In [204]:
# !pip install mysqlclient
# !pip install sqlalchemy

In [1]:
import pandas as pd
import os
import MySQLdb
import sqlalchemy

In [207]:
path = 'base_trabalho_pratico\\'
lst = []
for arq in os.listdir(path):
  a = os.path.join(path, arq)
  df = pd.read_csv(a, sep = '|', encoding = 'latin-1')
  lst.append(df)
df_pesquisa = pd.concat(lst)
df_pesquisa = df_pesquisa.reset_index()

In [208]:
df_pesquisa = df_pesquisa.rename(columns={'data_coleta':'data_pesquisa', 'bebida_favorita': 'bebida', 'hobbies': 'hobbie'})
df_pesquisa[:3]
# df_pequisa.describe()
# df_pequisa.info()
# df_pequisa.count()

,index,cod_pessoa,data_pesquisa,genero,data_nascimento,animal_estimacao,clima,bebida,hobbie
0,0,10141743,2020-01-01,Masculino,1981-04-25,peixe,frio,Cerveja,Praticar esporte
1,1,10141860,2020-01-01,Feminino,1966-01-19,peixe,quente,Café,Assistir TV
2,2,10142548,2020-01-01,Masculino,1973-08-02,tartaruga,frio,Café,Ler livros


In [209]:
df_pesquisa[['cod_pessoa', 'data_pesquisa', 'genero', 'data_nascimento', 'animal_estimacao', 'clima', 'bebida', 'hobbie']].to_csv('pesquisa.csv', sep = ';', encoding = 'latin-1', index = False)

In [222]:
df_pessoa = df_pesquisa[['cod_pessoa', 'genero', 'data_nascimento']].drop_duplicates()
lst_hobbie = list(df_pesquisa.hobbie.unique())
lst_clima = list(df_pesquisa.clima.unique())
lst_bebida = list(df_pesquisa.bebida.unique())
lst_animal_estimacao = list(df_pesquisa.animal_estimacao.unique())

In [211]:
usr = 'teste'
pwd = 'teste'
hst = '127.0.0.1'
dbs = 'teste'
str = f'mysql://{usr}:{pwd}@{hst}/{dbs}'

con = sqlalchemy.create_engine(str).connect()

In [212]:
def sql(query):
  try:
    con.execute(query)
  except Exception as e:
    print(f'Erro: {e} | Query: "{query}"')


In [213]:
dic_hobbie = {}
for cod, hobbie in enumerate(lst_hobbie):
  query = f'insert into hobbie values({cod + 1}, \'{hobbie}\');'
  sql(query)
  dic_hobbie[hobbie] = cod + 1

In [214]:
dic_clima = {}
for cod, clima in enumerate(lst_clima):
  query = f'insert into clima values({cod + 1}, \'{clima}\');'
  sql(query)
  dic_clima[clima] = cod + 1

In [215]:
dic_bebida = {}
for cod, bebida in enumerate(lst_bebida):
  query = f'insert into bebida values({cod + 1}, \'{bebida}\');'
  sql(query)
  dic_bebida[bebida] = cod + 1

In [216]:
dic_animal_estimacao = {}
for cod, animal_estimacao in enumerate(lst_animal_estimacao):
  query = f'insert into animal_estimacao values({cod + 1}, \'{animal_estimacao}\');'
  sql(query)
  dic_animal_estimacao[animal_estimacao] = cod + 1

In [223]:

for pessoa in df_pessoa.itertuples():
  query = f'insert into pessoa values({pessoa.cod_pessoa}, \'{pessoa.genero}\', \'{pessoa.data_nascimento}\');'
  sql(query)

In [217]:
# df_hobbie = pd.read_sql('hobbie', con = con)
# df_clima = pd.read_sql('clima', con = con)
# df_bebida = pd.read_sql('bebida', con = con)
# df_animal_estimacao = pd.read_sql('animal_estimacao', con = con)
df_pesq = df_pesquisa[['cod_pessoa', 'data_pesquisa', 'genero', 'data_nascimento', 'animal_estimacao', 'clima', 'bebida', 'hobbie']]
df_pesq.hobbie = df_pesquisa.hobbie.map(dic_hobbie)
df_pesq.clima = df_pesquisa.clima.map(dic_clima)
df_pesq.bebida = df_pesquisa.bebida.map(dic_bebida)
df_pesq.animal_estimacao = df_pesquisa.animal_estimacao.map(dic_animal_estimacao)
# df_pesq

c:\Users\tseii\WorkSpace\Python\coleta_dados\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [224]:
for pesq in df_pesq.itertuples():
  query = f'insert into pesquisa (data_pesquisa, cod_pessoa, cod_animal_estimacao, cod_bebida, cod_hobbie, cod_clima) values(\'{pesq.data_pesquisa}\', {pesq.cod_pessoa}, {pesq.animal_estimacao}, {pesq.bebida}, {pesq.hobbie}, {pesq.clima});'
  sql(query)